In [1]:
import re
import requests
import string
import numpy as np
import sympy as sp
from bs4 import BeautifulSoup

### Build the columns of our future document term matrix

In [2]:
# Scrape first link
response = requests.get("https://www.poetryfoundation.org/poems/51900/londons-summer-morning")
soup = BeautifulSoup(response.content, 'html.parser')
divs = soup.find_all('div', class_ = "poem-body overflow-x-auto")

# Clean divs
for div in divs:
    poem_text = div.text.lower()
    poem_text_no_punc = poem_text.translate(str.maketrans('', '', string.punctuation))
    poem = (poem_text_no_punc.split()) # Lower text and include no punctuation when we append

poem

['who',
 'has',
 'not',
 'waked',
 'to',
 'list',
 'the',
 'busy',
 'sounds',
 'of',
 'summer’s',
 'morning',
 'in',
 'the',
 'sultry',
 'smoke',
 'of',
 'noisy',
 'london',
 'on',
 'the',
 'pavement',
 'hot',
 'the',
 'sooty',
 'chimneyboy',
 'with',
 'dingy',
 'face',
 'and',
 'tattered',
 'covering',
 'shrilly',
 'bawls',
 'his',
 'trade',
 'rousing',
 'the',
 'sleepy',
 'housemaid',
 'at',
 'the',
 'door',
 'the',
 'milkpail',
 'rattles',
 'and',
 'the',
 'tinkling',
 'bell',
 'proclaims',
 'the',
 'dustman’s',
 'office',
 'while',
 'the',
 'street',
 'is',
 'lost',
 'in',
 'clouds',
 'impervious',
 'now',
 'begins',
 'the',
 'din',
 'of',
 'hackneycoaches',
 'waggons',
 'carts',
 'while',
 'tinmen’s',
 'shops',
 'and',
 'noisy',
 'trunkmakers',
 'knifegrinders',
 'coopers',
 'squeaking',
 'corkcutters',
 'fruitbarrows',
 'and',
 'the',
 'hungergiving',
 'cries',
 'of',
 'vegetablevendors',
 'fill',
 'the',
 'air',
 'now',
 'every',
 'shop',
 'displays',
 'its',
 'varied',
 'trade'

In [3]:
# Remove stopwords
stopword_list = ['the', 'and', 'is', 'in', 'to', 'of']
poem_no_stopword = [word for word in poem if word not in stopword_list]

### Build the rows of our future document term matrix

In this exercise I am going to define document by sentences of the poem which are split based on punctuation. Our first step in building the document term matrix is to define and pull our documents. 

In [4]:
response = requests.get("https://www.poetryfoundation.org/poems/51900/londons-summer-morning")
soup = BeautifulSoup(response.content, 'html.parser')
divs = soup.find_all('div', class_ = "poem-body overflow-x-auto")

# Clean divs
for div in divs:
    poem_text = div.text.lower()
    # Later note: Needed to remove parentheses since this messed up the sorted structure a few blocks down
    poem_text = re.split(r'[.!?()]', poem_text) # Use regex split so we can split by multiple punctuation types

poem_text # Now we have multiple topics "documents split by punctuation"

['who has not waked to list the busy sounds of summer’s morning, in the sultry smoke of noisy london',
 ' on the pavement hot the sooty chimney-boy, with dingy face and tattered covering, shrilly bawls his trade, rousing the sleepy housemaid',
 ' at the door the milk-pail rattles, and the tinkling bell proclaims the dustman’s office; while the street is lost in clouds impervious',
 ' now begins the din of hackney-coaches, waggons, carts; while tinmen’s shops, and noisy trunk-makers, knife-grinders, coopers, squeaking cork-cutters, fruit-barrows, and the hunger-giving cries of vegetable-vendors, fill the air',
 ' now every shop displays its varied trade, and the fresh-sprinkled pavement cools the feet of early walkers',
 ' at the private door the ruddy housemaid twirls the busy mop, annoying the smart ’prentice, or neat girl, tripping with band-box lightly',
 ' now the sun darts burning splendor on the glittering pane, save where the canvas awning throws a shade on the gay merchandise',

The next step is to build a vocabulary of unique words. We want to do this because we will use the unique words that appear in the dataset to construct our column index in the document-term matrix.

To do this we will use a "global vocabulary" and make a set of the unqiue words from the entire document.

In [5]:
unique_poem_set = set()

for sentence in poem_text:
    words = sentence.split() # Split the 13 sentences into words
    for word in words:
        unique_poem_set.update(words) # Create a set of words that and .update() the set for each iteration

In [6]:
unique_poem_sorted = sorted(unique_poem_set)
unique_poem_sorted

[',',
 'a',
 'abyss',
 'air',
 'all',
 'along',
 'and',
 'annoying',
 'area',
 'at',
 'awning',
 'bag',
 'band-box',
 'base',
 'bawls',
 'bears',
 'beauty',
 'begins',
 'bell',
 'burning',
 'busy',
 'canvas',
 'carts;',
 'catch',
 'charm',
 'chimney-boy,',
 'clouds',
 'cools',
 'coopers,',
 'cork-cutters,',
 'covering,',
 'cries',
 'dainties',
 'damsel;',
 'darts',
 'din',
 'dingy',
 'discordant',
 'displays',
 'domestic',
 'door',
 'dreams,',
 'dustman’s',
 'early',
 'enthrall',
 'every',
 'eye',
 'face',
 'feet',
 'fill',
 'for',
 'fresh-sprinkled',
 'from',
 'fruit-barrows,',
 'gay',
 'girl,',
 'glittering',
 'green',
 'hackney-coaches,',
 'half',
 'half-filled',
 'half-worn',
 'has',
 'his',
 'hot',
 'housemaid',
 'huge',
 'humming',
 'hunger-giving',
 'impervious',
 'in',
 'industry',
 'insects,',
 'is',
 'its',
 'knife-grinders,',
 'ladder,',
 'lamp-lighter',
 'lamps,',
 'lightly',
 'limy',
 'list',
 'load',
 'london',
 'lost',
 'merchandise',
 'milk-pail',
 'minute',
 'monotonou

### Build the document-term matrix

Observation: The document-term matrix should have dimensions number of documents x number of words so I will take len() of these before I initialize the zero matrix and begin updating entries. 

In [7]:
# Check lengths

len(unique_poem_sorted) # 188 words
len(poem_text) # 17 documents

17

In [8]:
# Initialize a 17 x 187 zero matrix
dtm = np.zeros((17, 188))


# Make a lookup dictionary for the words (O(1) lookup in best case)
# We know this lookup only contains words in our unique_poem_sorted
word_lookup = { v : k for k, v in enumerate(unique_poem_sorted)}

for d_idx, document in enumerate(poem_text):
    for word in document.split():
        word_idx = word_lookup[word]
        dtm[d_idx][word_idx] += 1
dtm

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 1., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [9]:
assert unique_poem_sorted[182] == 'who'

In [10]:
# Check that the word 'who' has a 1 in the document 0, entry 182
assert dtm[0][182] == 1

In [11]:
# Use TF formula (from Wikipedia) to normalize word frequencies
# This sums row-wise
row_sums = np.sum(dtm, axis = 1, keepdims=True)

# This np.divide method ensures we skip over rows with zeros and prevent division by zero errors
#tf_matrix = np.divide(dtm, row_sums, where = row_sums != 0)
# Add the tiny constant 1e-10 to avoid Runtime Warnings about division with zero

tf_matrix = np.where(row_sums != 0, np.divide(dtm, row_sums + 1e-10), 0)
tf_matrix

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.09090909, 0.        , 0.09090909, ..., 0.09090909, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [12]:
# IDF looks at the the number of documents where a word appears so we sum across the columns
col_sums = np.sum(dtm, axis = 0)

# IDF smooth formula (from Wikipedia page on tf-idf)
idf_matrix = np.log(np.divide(len(poem_text), (1 + col_sums), where= col_sums != 0)) + 1
idf_matrix

array([3.14006616, 3.14006616, 3.14006616, 3.14006616, 3.14006616,
       2.73460106, 1.63598877, 3.14006616, 3.14006616, 2.44691898,
       3.14006616, 3.14006616, 3.14006616, 3.14006616, 3.14006616,
       3.14006616, 3.14006616, 3.14006616, 3.14006616, 2.73460106,
       2.44691898, 3.14006616, 3.14006616, 3.14006616, 3.14006616,
       3.14006616, 3.14006616, 3.14006616, 3.14006616, 3.14006616,
       3.14006616, 2.73460106, 3.14006616, 3.14006616, 3.14006616,
       3.14006616, 3.14006616, 3.14006616, 3.14006616, 3.14006616,
       2.73460106, 3.14006616, 3.14006616, 3.14006616, 3.14006616,
       2.73460106, 3.14006616, 3.14006616, 2.73460106, 3.14006616,
       2.73460106, 3.14006616, 3.14006616, 3.14006616, 3.14006616,
       3.14006616, 3.14006616, 3.14006616, 3.14006616, 3.14006616,
       3.14006616, 3.14006616, 3.14006616, 2.22377543, 3.14006616,
       2.73460106, 3.14006616, 3.14006616, 3.14006616, 3.14006616,
       2.04145387, 3.14006616, 3.14006616, 2.73460106, 2.73460

In [13]:
# Element-wise multiplication of the tf_matrix and idf_matrix
# Wikipedia page includes the tf-idf formula

tf_idf_matrix = tf_matrix * idf_matrix
tf_idf_matrix

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.28546056, 0.        , 0.28546056, ..., 0.28546056, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [101]:
# Redefine tf_idf_matrix as A

A = tf_idf_matrix

# SVD step 1: Compute eigenthings of ATA
A_T = np.transpose(A)
ATA = np.matmul(A_T, A)

# Unpack the tuple from .eig
ATA_eigenvalues, ATA_eigenvectors = np.linalg.eig(ATA)

# SVD step 2: Compute eigenthings of AAT
AAT = np.matmul(A, A_T)
AAT_eigenvalues, AAT_eigenvectors = np.linalg.eig(AAT)

# There is a theorem that tells us that the non-zero eigenvalues are the same between AAT and ATA
# By inspection we can see that the first 16 eigenvalues of ATA are non-zero
# We also know that the rank between ATA and AAT is the same since the # of non-zero eigenvalues is the same
ATA_nonzero_eig = ATA_eigenvalues[:16]
ATA_nonzero_eig

AAT_nonzero_eig = AAT_eigenvalues[:16]

assert ATA_nonzero_eig.all() == AAT_nonzero_eig.all() # We confirm that the non-zero eigenvalues are equal
assert np.linalg.matrix_rank(AAT) == np.linalg.matrix_rank(ATA)

# SVD step 3: Build U, V, ∑ from the matrix information

sigma = np.sqrt(AAT_nonzero_eig) # Already asserted these are the same eigenvalues as ATA. Take sqrt since we have ∑^2 when we do ATA or AAT
sigma_matrix = np.diag(sigma) # Use np.diag so we get a matrix for sigma instead of a 16 x 1 vector

# Check orthogonality of eigenvectors in AAT
U_orthocheck = np.dot(AAT_eigenvectors[0], AAT_eigenvectors[1]) # Answer: 1.3624863072118684e-15
U = AAT_eigenvectors[:16]

V_orthocheck = np.dot(ATA_eigenvectors[0], ATA_eigenvectors[1]) # Answer: 0.0064162921966990204-7.806255641895632e-18j
V_orthocheck

# Because V_orthocheck is not as close to zero as U_orthocheck check the eigenvalues of the first eigenvector to see how close they are
# Conclusion: They are very close. Good to move on.
AAT_eigenvalues[0] # Answer: 1.7298486774104262
ATA_eigenvalues[0] # Answer: 1.7298486774104356+0j

V = ATA_eigenvectors[:16]

# Selects all rows and all columns except for the last one - the last column had all zeros so it was incorrect
U_truncated = U[:, :-1] 

assert U_truncated.shape == (16, 16) # Confirm the shape is m x r (r = rank)
assert sigma_matrix.shape == (16, 16) # Confirm the shape is r x r (r = rank). We should only have as many singular values as our rank.
assert V.shape == (16, 188) # Confirm the shape is r x n (r = rank)

(16, 16)

In [102]:
# Construct a rank 16 approximation of A

A_int = np.matmul(U_truncated, sigma_matrix)
A = np.matmul(A_int, V)
A.shape

(16, 188)